#### 7/1　選定したringで学習

In [1]:
import pandas as pd
from math import sqrt as sqrt
from itertools import product as product

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
from torchsummary import summary
import torch.utils.data as data
import torch.optim as optim

import ast
import cv2
import numpy as np
import time
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [2]:
from utils.ssd_model import SSD
from utils.ssd_model import MultiBoxLoss

In [3]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, path, patience=7, verbose=False, delta=0, trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

        

In [4]:
input_size = 300
color_mean = (0, 0)
voc_classes = ['ring']

In [5]:
ssd_cfg = {
    'num_classes': 2,  # 背景クラスを含めた合計クラス数
    'input_size': input_size,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ   
#     'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

### <font color="Red">fine tunning</font>

In [22]:
net = SSD(phase='train', cfg=ssd_cfg)
CNN_weight = torch.load('ssd300_mAP_77.43_v2.pth')

In [23]:
net.vgg[0].weight = nn.Parameter(CNN_weight['vgg.0.weight'][:,:2,:,:])#.detach().numpy().copy()
net.vgg[0].bias = nn.Parameter(CNN_weight['vgg.0.bias'])
net.vgg[2].weight = nn.Parameter(CNN_weight['vgg.2.weight'])
net.vgg[2].bias = nn.Parameter(CNN_weight['vgg.2.bias'])
net.vgg[5].weight = nn.Parameter(CNN_weight['vgg.5.weight'])
net.vgg[5].bias = nn.Parameter(CNN_weight['vgg.5.bias'])
net.vgg[7].weight = nn.Parameter(CNN_weight['vgg.7.weight'])
net.vgg[7].bias = nn.Parameter(CNN_weight['vgg.7.bias'])
net.vgg[10].weight = nn.Parameter(CNN_weight['vgg.10.weight'])
net.vgg[10].bias = nn.Parameter(CNN_weight['vgg.10.bias'])
net.vgg[12].weight = nn.Parameter(CNN_weight['vgg.12.weight'])
net.vgg[12].bias = nn.Parameter(CNN_weight['vgg.12.bias'])
net.vgg[14].weight = nn.Parameter(CNN_weight['vgg.14.weight'])
net.vgg[14].bias = nn.Parameter(CNN_weight['vgg.14.bias'])
net.vgg[17].weight = nn.Parameter(CNN_weight['vgg.17.weight'])
net.vgg[17].bias = nn.Parameter(CNN_weight['vgg.17.bias'])
net.vgg[19].weight = nn.Parameter(CNN_weight['vgg.19.weight'])
net.vgg[19].bias = nn.Parameter(CNN_weight['vgg.19.bias'])
net.vgg[21].weight = nn.Parameter(CNN_weight['vgg.21.weight'])
net.vgg[21].bias = nn.Parameter(CNN_weight['vgg.21.bias'])
net.vgg[24].weight = nn.Parameter(CNN_weight['vgg.24.weight'])
net.vgg[24].bias = nn.Parameter(CNN_weight['vgg.24.bias'])
net.vgg[26].weight = nn.Parameter(CNN_weight['vgg.26.weight'])
net.vgg[26].bias = nn.Parameter(CNN_weight['vgg.26.bias'])
net.vgg[28].weight = nn.Parameter(CNN_weight['vgg.28.weight'])
net.vgg[28].bias = nn.Parameter(CNN_weight['vgg.28.bias'])
net.vgg[31].weight = nn.Parameter(CNN_weight['vgg.31.weight'])
net.vgg[31].bias = nn.Parameter(CNN_weight['vgg.31.bias'])
net.vgg[33].weight = nn.Parameter(CNN_weight['vgg.33.weight'])
net.vgg[33].bias = nn.Parameter(CNN_weight['vgg.33.bias'])

In [24]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

SSD(
  (vgg): ModuleList(
    (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

In [25]:
summary(net, (2, 300, 300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 300, 300]           1,216
              ReLU-2         [-1, 64, 300, 300]               0
            Conv2d-3         [-1, 64, 300, 300]          36,928
              ReLU-4         [-1, 64, 300, 300]               0
         MaxPool2d-5         [-1, 64, 150, 150]               0
            Conv2d-6        [-1, 128, 150, 150]          73,856
              ReLU-7        [-1, 128, 150, 150]               0
            Conv2d-8        [-1, 128, 150, 150]         147,584
              ReLU-9        [-1, 128, 150, 150]               0
        MaxPool2d-10          [-1, 128, 75, 75]               0
           Conv2d-11          [-1, 256, 75, 75]         295,168
             ReLU-12          [-1, 256, 75, 75]               0
           Conv2d-13          [-1, 256, 75, 75]         590,080
             ReLU-14          [-1, 256,